In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv("../../data/data/train-metadata.csv", low_memory=False)
missing_percentages = (df.isnull().sum() / len(df)) * 100
columns_to_keep = missing_percentages[missing_percentages < 50].index
skin_cancer_df = df[columns_to_keep]
skin_cancer_df = skin_cancer_df.fillna(skin_cancer_df.mode().iloc[0])


In [ ]:
# Label encode all categorical columns
categorical_columns = skin_cancer_df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    skin_cancer_df[col] = le.fit_transform(skin_cancer_df[col])

X = skin_cancer_df.drop('target', axis=1)
y = skin_cancer_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

MemoryError: Unable to allocate 150. GiB for an array with shape (401059, 401059) and data type bool

In [ ]:

rf = RandomForestClassifier(n_estimators=300, class_weight='balanced', random_state=42)
rf.fit(X_train_sm, y_train_sm)

accuracy = rf.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

y_pred = rf.predict(X_test) 
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))